In [4]:
import pandas as pd
import pickle

In [97]:
import unicodedata

#removes special characters such as "é,ñ" for example "Michael Bublé" to "Michael Buble"
def getStringWithoutSpecialAccent(string):
    return ''.join((c for c in unicodedata.normalize('NFD', string) if unicodedata.category(c) != 'Mn'))

In [73]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [6]:
songs = pd.read_pickle('songsWeeklyRanks.pkl')

In [19]:
songs["SpotifyID"] = None

In [20]:
#dictionary with key "Fine Line-Harry Styles" with value "SpotifyID"

In [13]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid = "2db050f400f245738228ef384306044b"
secret = "6309c1f0455b43428d2cf142e4130a64"

client_credentials_manager = SpotifyClientCredentials(client_id = cid, client_secret = secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [17]:
def getSongID(trackName, trackArtist, trackResults):
    for track in trackResults['tracks']['items']:
        if (track['name'].lower() == trackName.lower()) & (track['artists'][0]['name'].lower() == trackArtist.lower()):
            return track['id']
    return None

def getAlbumID(albumName, albumArtist, searchResult):
    #TODO: HERE HERE HERE HERE
    #NEED TO DEAL WITH SPECIAL CHARACTERS WHEN CHECKING NAMES
    #NEED TO DEAL WITH VARIATIONS SUCH AS (DELUXE EDITION)
    getStringWithoutSpecialAccent(album['name'].lower())
    getStringWithoutSpecialAccent(albumArtist.lower())
        
    getStringWithoutSpecialAccent(album['artists'][0]['name'].lower())
    getStringWithoutSpecialAccent(albumArtist.lower())
    for album in searchResult['albums']['items']:
        if (album['name'].lower() == albumName.lower()) & (album['artists'][0]['name'].lower() == albumArtist.lower()):
            return album['id']
    return None

def getAlbumSongs(albumArtist, albumTracksResults):
    albumSongDF = pd.DataFrame(columns=["Title","Artist", "SpotifyID"])
    for songs in albumTracksResults['items']:
        newRow = pd.Series(data={"Title": songs['name'],"Artist": albumArtist, "SpotifyID": songs['id']})
        albumSongDF = albumSongDF.append(newRow, ignore_index=True)
    return albumSongDF

In [75]:
##create two df
songsToDeleteDF = pd.DataFrame(columns=["Title","Artist", "SpotifyID"])
albumSongToAddDF = pd.DataFrame(columns=["Date","Title","Artist","Rank", "SpotifyID"])

spotifyIDForSongsCache = {} #stores spotify ids for songs that you already queried
songsDSForAlbumCache = {} #stores DataFrame of Songs (Title,Artist,SpotifyID) from an Album

#iterate through all song rows and find SpotifyID
for index, row in songs.iterrows(): 
    songNameAndArtist = row['Title'] + "-" + row['Artist'] #used for key values on caches
    if spotifyIDForSongsCache.get(songNameAndArtist) is not None: #if it exists in cache, use that instead
        #print(songNameAndArtist + " Used Song Cache")
        songID = spotifyIDForSongsCache[songNameAndArtist]
        songs.SpotifyID.iloc[index] = songID
        continue
    if songsDSForAlbumCache.get(songNameAndArtist) is not None: #if it exists in cache, use that instead
        #print(songNameAndArtist + " Used Album Cache")
        songsFromAlbum = songsDSForAlbumCache[songNameAndArtist]
        songsFromAlbum["Date"] = row["Date"]
        songsFromAlbum["Rank"] = row["Rank"]
        albumSongToAddDF = pd.concat([albumSongToAddDF,songsFromAlbum], ignore_index=True)
        continue
    
    #get songID from Spotify Search API
    trackResults = sp.search(q=row['Title'], type='track', market= 'US', limit=10,offset=0)
    songID = getSongID(row['Title'], row['Artist'], trackResults)
    #If songID is not found, we assume it is an album and search for its albumID
    if  songID == None:
        #Delete the row from songs DF that does not have spotifyID since they are an album
        songsToDeleteDF = songsToDeleteDF.append(row, ignore_index=True)
        searchResult = sp.search(q=row['Title'], type='album', market= 'US', limit=10,offset=0)
        albumID = getAlbumID(row['Title'], row['Artist'], searchResult)
        #If we find an albumID, we get all the songs from that album and add it to the DF
        if albumID != None:
            albumTracksResults = sp.album_tracks(albumID)
            songsFromAlbum = getAlbumSongs(row['Artist'],albumTracksResults)
            #cache
            keyToStore = songNameAndArtist
            songsDSForAlbumCache[keyToStore] = songsFromAlbum
            #add date and rank
            songsFromAlbum["Date"] = row["Date"]
            songsFromAlbum["Rank"] = row["Rank"]
            
            albumSongToAddDF = pd.concat([albumSongToAddDF,songsFromAlbum], ignore_index=True)
    else:
        #Add the spotifyID to an existing song
        #cache 
        keyToStore = songNameAndArtist
        spotifyIDForSongsCache[keyToStore] = songID
        
        songs.SpotifyID.iloc[index] = songID


In [76]:
songs["Rank"] = songs["Rank"].astype(int)
songs = pd.concat([songs,albumSongToAddDF],ignore_index=True) #add the album songs into the song DF
songs = songs[~songs.Title.isin(songsToDeleteDF.Title)].copy() #remove album or song without spotifyID from songs DF
songs.head()

,Date,Title,Artist,Rank,SpotifyID
0,2020-01-01,Fine Line,Harry Styles,1,6VzcQuzTNTMFnJ6rBSaLH9
5,2020-01-01,Hollywood's Bleeding,Post Malone,6,7sWRlDoTDX8geTR8zzr2vt
10,2020-01-01,Lover,Taylor Swift,11,1dGr1c8CrMLDpV6mPbImSI
15,2020-01-01,Rudolph The Red-Nosed Reindeer,Burl Ives,16,47otoIkhx3fkdivEXL5OB6
18,2020-01-01,What You See Is What You Get,Luke Combs,19,273TiTHLlHSRe5zrzs7wvD


In [91]:
songs.sort_values(by=["Date", "Rank"],inplace=False).head(3000) #sorting the DF by date and rank

,Date,Title,Artist,Rank,SpotifyID
0,2020-01-01,Fine Line,Harry Styles,1,6VzcQuzTNTMFnJ6rBSaLH9
10600,2020-01-01,Intro,Roddy Ricch,3,7AOtINPzQ4N4JVXk7ygZZz
10601,2020-01-01,The Box,Roddy Ricch,3,0nbXyq5TXYPCO7pr3N8S4I
10602,2020-01-01,Start Wit Me (feat. Gunna),Roddy Ricch,3,7A20WnLkjtZU2d7hIB6Ziu
10603,2020-01-01,Perfect Time,Roddy Ricch,3,6kiIVIbmwEw6JvIZc7UG0E
10604,2020-01-01,Moonwalkin (feat. Lil Durk),Roddy Ricch,3,33NoCir8q4WA4HNmyrD3fu
10605,2020-01-01,Big Stepper,Roddy Ricch,3,6x0QNM6f0C2JEjK0hFF8Ca
10606,2020-01-01,Gods Eyes,Roddy Ricch,3,7KK4d7xfH6WjUuK1kPL8E2
10607,2020-01-01,Peta (feat. Meek Mill),Roddy Ricch,3,4bl1HBZliBuYpFZjWgefE4
10608,2020-01-01,Boom Boom Room,Roddy Ricch,3,3aBL75wBGHJnskegE9jyhb


In [93]:
songs.loc[songs.Artist=="Michael Buble"] #to test if MB is in the DF

,Date,Title,Artist,Rank,SpotifyID
